# 如何从头开始创建 ReAct 代理（函数式 API）

!!!信息“先决条件”
本指南假设您熟悉以下内容：

- [聊天模型](https://js.langchain.com/docs/concepts/chat_models)
- [消息](https://js.langchain.com/docs/concepts/messages)
- [工具调用](https://js.langchain.com/docs/concepts/tool_calling/)
- [入口点](../../concepts/tical_api/#entrypoint) 和 [任务](../../concepts/tical_api/#task)

本指南演示如何使用 LangGraph [Functional API](../../concepts/functional_api) 实现 ReAct 代理。

ReAct 代理是一个[工具调用代理](../../concepts/agentic_concepts/#tool-calling-agent)，其操作如下：

1.向聊天模型发出查询；
2. 如果模型没有生成[工具调用](../../concepts/agentic_concepts/#tool-calling)，我们返回模型响应。
3. 如果模型生成工具调用，我们使用可用工具执行工具调用，将它们作为[工具消息](https://js.langchain.com/docs/concepts/messages/)附加到我们的消息列表中，然后重复该过程。

这是一种简单且多功能的设置，可以通过内存、人机交互功能和其他功能进行扩展。有关示例，请参阅专用的[操作指南](../../how-tos/#prebuilt-react-agent)。

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/openai @langchain/core zod
```
接下来，我们需要为 OpenAI 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 创建ReAct代理

现在您已经安装了所需的软件包并设置了环境变量，我们可以创建代理。

### 定义模型和工具

让我们首先定义我们将用于示例的工具和模型。在这里，我们将使用一个占位符工具来获取某个位置的天气描述。

在本示例中，我们将使用 [OpenAI](https://js.langchain.com/docs/integrations/providers/openai/) 聊天模型，但任何[支持工具调用](https://js.langchain.com/docs/integrations/chat/) 的模型就足够了。

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const getWeather = tool(async ({ location }) => {
  const lowercaseLocation = location.toLowerCase();
  if (lowercaseLocation.includes("sf") || lowercaseLocation.includes("san francisco")) {
    return "It's sunny!";
  } else if (lowercaseLocation.includes("boston")) {
    return "It's rainy!";
  } else {
    return `I am not sure what the weather is in ${location}`;
  }
}, {
  name: "getWeather",
  schema: z.object({
    location: z.string().describe("location to get the weather for"),
  }),
  description: "Call to get the weather from a specific location."
});

const tools = [getWeather];

### 定义任务

接下来我们定义我们将执行的 [tasks](../../concepts/tical_api/#task)。这里有两个不同的任务：

1. **调用模型**：我们想要使用消息列表查询我们的聊天模型。
2. **调用工具**：如果我们的模型生成工具调用，我们想要执行它们。

In [2]:
import {
  type BaseMessageLike,
  AIMessage,
  ToolMessage,
} from "@langchain/core/messages";
import { type ToolCall } from "@langchain/core/messages/tool";
import { task } from "@langchain/langgraph";

const toolsByName = Object.fromEntries(tools.map((tool) => [tool.name, tool]));

const callModel = task("callModel", async (messages: BaseMessageLike[]) => {
  const response = await model.bindTools(tools).invoke(messages);
  return response;
});

const callTool = task(
  "callTool",
  async (toolCall: ToolCall): Promise<AIMessage> => {
    const tool = toolsByName[toolCall.name];
    const observation = await tool.invoke(toolCall.args);
    return new ToolMessage({ content: observation, tool_call_id: toolCall.id });
    // 也可以直接将toolCall传递到工具中以返回ToolMessage
    // 返回 tool.invoke(toolCall);
  });

### 定义入口点

我们的[entrypoint](../../concepts/tical_api/#entrypoint)将处理这两个任务的编排。如上所述，当我们的 `callModel` 任务生成工具调用时，`callTool` 任务将为每个工具生成响应。我们将所有消息附加到单个消息列表中。

In [3]:
import { entrypoint, addMessages } from "@langchain/langgraph";

const agent = entrypoint(
  "agent",
  async (messages: BaseMessageLike[]) => {
    let currentMessages = messages;
    let llmResponse = await callModel(currentMessages);
    while (true) {
      if (!llmResponse.tool_calls?.length) {
        break;
      }

      // 执行工具
      const toolResults = await Promise.all(
        llmResponse.tool_calls.map((toolCall) => {
          return callTool(toolCall);
        })
      );
      
      // 附加到消息列表
      currentMessages = addMessages(currentMessages, [llmResponse, ...toolResults]);

      // 再次调用模型
      llmResponse = await callModel(currentMessages);
    }

    return llmResponse;
  }
);

## 用法

要使用我们的代理，我们使用消息列表来调用它。根据我们的实现，这些可以是 LangChain [消息](https://js.langchain.com/docs/concepts/messages/) 对象或 OpenAI 风格的对象：

In [4]:
import { BaseMessage, isAIMessage } from "@langchain/core/messages";

const prettyPrintMessage = (message: BaseMessage) => {
  console.log("=".repeat(30), `${message.getType()} message`, "=".repeat(30));
  console.log(message.content);
  if (isAIMessage(message) && message.tool_calls?.length) {
    console.log(JSON.stringify(message.tool_calls, null, 2));
  }
}

// 使用示例
const userMessage = { role: "user", content: "What's the weather in san francisco?" };
console.log(userMessage);

const stream = await agent.stream([userMessage]);

for await (const step of stream) {
  for (const [taskName, update] of Object.entries(step)) {
    const message = update as BaseMessage;
    // 仅打印任务更新
    if (taskName === "agent") continue;
    console.log(`\n${taskName}:`);
    prettyPrintMessage(message);
  }
}

{ role: 'user', content: "What's the weather in san francisco?" }

callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_m5jZoH1HUtH6wA2QvexOHutj"
  }
]

callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


完美的！该图正确调用`getWeather`工具，并在收到工具发送的信息后响应用户。查看 LangSmith 跟踪[此处](https://smith.langchain.com/public/8132d3b8-2c91-40fc-b660-b766ca33e9cb/r)。

## 添加线程级持久化

添加[线程级持久性](../../concepts/persistence#threads) 让我们支持与代理的对话体验：后续调用将附加到先前的消息列表，保留完整的对话上下文。

向我们的代理添加线程级持久性：

1. 选择一个[checkpointer](../../concepts/persistence#checkpointer-libraries)：这里我们将使用[MemorySaver](/langgraphjs/reference/classes/checkpoint.MemorySaver.html)，一个简单的内存检查指针。
2. 更新我们的入口点以接受先前的消息状态作为第二个参数。在这里，我们只是将消息更新附加到之前的消息序列中。
3. 选择将从工作流程返回哪些值以及由检查点保存哪些值。如果我们从 `entrypoint.final` 返回它，我们将能够以 `getPreviousState()` 的形式访问它（可选）

In [5]:
import {
  MemorySaver,
  getPreviousState,
} from "@langchain/langgraph";

// 突出显示下一行
const checkpointer = new MemorySaver();

const agentWithMemory = entrypoint({
  name: "agentWithMemory",
  // 突出显示下一行
  checkpointer,
}, async (messages: BaseMessageLike[]) => {
  const previous = getPreviousState<BaseMessage>() ?? [];
  let currentMessages = addMessages(previous, messages);
  let llmResponse = await callModel(currentMessages);
  while (true) {
    if (!llmResponse.tool_calls?.length) {
      break;
    }

    // 执行工具
    const toolResults = await Promise.all(
      llmResponse.tool_calls.map((toolCall) => {
        return callTool(toolCall);
      })
    );
    
    // 附加到消息列表
    currentMessages = addMessages(currentMessages, [llmResponse, ...toolResults]);

    // 再次调用模型
    llmResponse = await callModel(currentMessages);
  }
  
  // 附加最终响应以进行存储
  currentMessages = addMessages(currentMessages, llmResponse);

  // 突出显示下一行
  return entrypoint.final({
    // 突出显示下一行
    value: llmResponse,
    // 突出显示下一行
    save: currentMessages,
    // 突出显示下一行
  });
});

现在，我们需要在运行应用程序时传递配置。该配置将为会话线程指定一个标识符。

!!!提示

在我们的[概念页面](../../concepts/persistence/) 和[操作指南](../../how-tos/#persistence) 中了解有关线程级持久性的更多信息。

In [6]:
const config = { configurable: { thread_id: "1" } };

我们以与之前相同的方式启动一个线程，这次传入配置：

In [7]:
const streamWithMemory = await agentWithMemory.stream([{
  role: "user",
  content: "What's the weather in san francisco?",
}], config);

for await (const step of streamWithMemory) {
  for (const [taskName, update] of Object.entries(step)) {
    const message = update as BaseMessage;
    // 仅打印任务更新
    if (taskName === "agentWithMemory") continue;
    console.log(`\n${taskName}:`);
    prettyPrintMessage(message);
  }
}


callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "san francisco"
    },
    "type": "tool_call",
    "id": "call_4vaZqAxUabthejqKPRMq0ngY"
  }
]

callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


当我们询问后续对话时，模型使用先前的上下文来推断我们正在询问天气：

In [8]:
const followupStreamWithMemory = await agentWithMemory.stream([{
  role: "user",
  content: "How does it compare to Boston, MA?",
}], config);

for await (const step of followupStreamWithMemory) {
  for (const [taskName, update] of Object.entries(step)) {
    const message = update as BaseMessage;
    // 仅打印任务更新
    if (taskName === "agentWithMemory") continue;
    console.log(`\n${taskName}:`);
    prettyPrintMessage(message);
  }
}


callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "boston, ma"
    },
    "type": "tool_call",
    "id": "call_YDrNfZr5XnuBBq5jlIXaxC5v"
  }
]

callTool:
============================== tool message ==============================
It's rainy!

callModel:
============================== ai message ==============================
In comparison, while San Francisco is sunny, Boston, MA is experiencing rain.


在 [LangSmith 跟踪](https://smith.langchain.com/public/ec803712-ecfc-49b6-8f54-92252d1e5e33/r) 中，我们可以看到每个模型调用中都保留了完整的对话上下文。